In [1]:
import keras
from keras import layers
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from autograd import grad
import theano
import autograd.numpy as np

import os

import tensorflow as tf
from tensorflow import keras


import pymc3 as pm
import theano.tensor as T

In [16]:
def softmax1(x):
  return np.exp(x) / np.sum(np.exp(x), axis=1)[:,None]

In [4]:
X, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)


# Pick out 3 classes of digits: 0, 1, 2 and take a subset of samples as in-distribution points
X_0, y_0 = X[(y == '0')][:500], y[(y == '0')][:500].astype(int)
X_1, y_1 = X[(y == '1')][:500], y[(y == '1')][:500].astype(int)
X_2, y_2 = X[(y == '2')][:500], y[(y == '2')][:500].astype(int)

X_ood, y_ood = X[(y == '3')][:500], ['OOD'] * 500

In [11]:
# combine data
X_mnist_ood = np.concatenate((X_0, X_1, X_2,X_ood))
y_mnist_ood = np.concatenate((y_0, y_1, y_2,y_ood))

X_mnist = np.concatenate((X_0, X_1, X_2))
y_mnist = np.concatenate((y_0, y_1, y_2))


In [12]:
def MNIST_NN(X,Y, input_shape, nc, batch_size, epochs, loss):
    # separate data
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, 
                                                        stratify = Y, random_state=0)
    x_train = np.array(x_train)
    y_train = np.array(pd.get_dummies(y_train))
    x_test = np.array(x_test)
    y_test = np.array(pd.get_dummies(y_test))
    print("Shape of data: ", x_train.shape,y_train.shape, x_test.shape,y_test.shape)
    # train the model  
    batch_size = batch_size
    epochs = epochs
    
    model = make_model(input_shape=input_shape, num_classes=nc)
    model.compile(loss = loss, optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,verbose=0)
    # evaluation 
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    pred = model.predict(x_test)
    print(pred.shape)
    auc = roc_auc_score(y_test, pred)
    print("AUC: ", auc)
    return model 

model = MNIST_NN(X_mnist_ood,y_mnist_ood, 784, 3+1, 32, 15,  "categorical_crossentropy")
intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(index = 4).output)

intermediate_output = np.array(intermediate_layer_model(np.array(X_mnist)))


Shape of data:  (1400, 784) (1400, 4) (600, 784) (600, 4)
Test loss: 0.42664089798927307
Test accuracy: 0.95333331823349
(600, 4)
AUC:  0.9923240740740741


In [13]:
intermediate_output.shape

(1500, 12)

In [14]:
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
nc = 3 # 3 classes
D = nc * (intermediate_output.shape[1] + 1) 
x = intermediate_output
y = np.array(y_mnist)
Priord = 1
x_with_constant = np.hstack((x, np.ones((x.shape[0], 1), dtype=x.dtype)))
with pm.Model() as bayesian_model:
    #define priors
    W = pm.Normal('W', mu=0, tau=1./(Priord**2), shape= D)
    
    p = pm.math.dot(x_with_constant, W.reshape((-1,nc))) 
    
    #softmax
    theta = T.nnet.softmax(p)
    #define binomial likelihood
    y_obs = pm.Categorical('y_obs',  p=theta, observed=y)
    step = pm.Metropolis()
    trace = pm.sample(5000,step,tune = 10000,chains = 2)
thinned_trace = trace[::1]['W']

Sequential sampling (2 chains in 1 job)
Metropolis: [W]
100%|██████████| 15000/15000 [28:39<00:00,  8.72it/s]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [17]:
intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.get_layer(index = 4).output)

output_in = np.array(intermediate_layer_model(np.array(X_mnist)))
output_ood = np.array(intermediate_layer_model(np.array(X_ood)))
in_withc = np.hstack((output_in,np.ones((output_in.shape[0],1),dtype = output_in.dtype)))
ood_withc = np.hstack((output_ood,np.ones((output_ood.shape[0],1),dtype = output_ood.dtype)))

#calculate uncertainty(using variance)
rand_samples = thinned_trace #[:700] #changed depending on sample!!!!!
all_preds_in = []
all_preds_ood = []
for W in rand_samples:
    all_preds_in.append(softmax1(in_withc @ np.array(W).reshape((-1,nc))))
    all_preds_ood.append(softmax1(ood_withc @ np.array(W).reshape((-1,nc))))
#assign predicted labels
mean_label_in = np.argmax(np.mean(all_preds_in,axis = 0),axis = 1)
mean_label_ood = np.argmax(np.mean(all_preds_ood,axis = 0),axis = 1)

#assign uncertainty (using variance)
varss_in = np.var(all_preds_in,axis = 0)
var_in= np.mean([varss_in[i][mean_label_in[i]] for i in range(len(varss_in))])

varss_ood = np.var(all_preds_ood,axis = 0)
var_ood= np.mean([varss_ood[i][mean_label_ood[i]] for i in range(len(varss_ood))])  

print("Uncertainty for in distribution data is ",var_in)
print("Uncertainty for ood data is ", var_ood)

Uncertainty for in distribution data is  1.00503935694972e-06
Uncertainty for ood data is  6.4134650907512935e-06
